<a href="https://colab.research.google.com/github/2019zhou/helperFunction/blob/master/DataPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training using fairseq
- 一个使用bug数据集, fairseq框架跑的一个模型
- 基本上所有参数都还没有进行任何改动，使用bpe
- 使用了完全nlp方式来训练code数据
- 主要目的在于查看这样训练的可行性

## 下载两个对应的数据集
- src-all.txt
- tgt-all.txt
- 注意使用 -q 不然输出结果太大，colab会有报错


In [ ]:
%%bash
wget -q https://zenodo.org/record/4067881/files/src-all.txt

In [ ]:
%%bash
wget -q https://zenodo.org/record/4067881/files/tgt-all.txt

In [ ]:
# 把数据拷贝到google云盘中
#!cp '/content/src-all.txt' -r '/content/drive/My Drive/bugData/'
!cp '/content/tgt-all.txt' -r '/content/drive/My Drive/bugData/'

In [ ]:
# 如果上一次已经拷贝到了google drive中，这次只需要从google drive中把数据拷贝出来就可以了,里面有srcoutput tgtoutput
!cp '/content/drive/My Drive/bugData/' -r '/content/data'

## Explore the dataset

In [ ]:
%%bash
head -n 50 src-all.txt

void cpuidle_enter_freeze ( void ) { struct cpuidle_device * dev = __this_cpu_read ( cpuidle_devices ) ; struct cpuidle_driver * drv = cpuidle_get_cpu_driver ( dev ) ; int index ; if ( cpuidle_not_available ( drv , dev ) ) goto fallback ; index = cpuidle_find_deepest_state ( drv , dev , true ) ; if ( index >= 0 ) { enter_freeze_proper ( drv , dev , index ) ; local_irq_enable ( ) ; return ; } index = cpuidle_find_deepest_state ( drv , dev , false ) ; if ( index >= 0 ) { cpuidle_enter ( drv , dev , index ) ; return ; } fallback : arch_cpu_idle ( ) ; }
int cpuidle_select ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { if ( cpuidle_not_available ( drv , dev ) ) return - ENODEV ; return cpuidle_curr_governor -> select ( drv , dev ) ; }
static bool cpuidle_not_available ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { return off || ! initialized || ! drv || ! dev || ! dev -> enabled ; }
static int cpuidle_find_deepest_state ( struct cpuidle_driver * drv , struc

In [ ]:
%%bash 
head -n 50 tgt-all.txt

int cpuidle_enter_freeze ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { int index ; index = find_deepest_state ( drv , dev , true ) ; if ( index >= 0 ) enter_freeze_proper ( drv , dev , index ) ; return index ; }
int cpuidle_select ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { return cpuidle_curr_governor -> select ( drv , dev ) ; }
bool cpuidle_not_available ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { return off || ! initialized || ! drv || ! dev || ! dev -> enabled ; }
int cpuidle_find_deepest_state ( struct cpuidle_driver * drv , struct cpuidle_device * dev ) { return find_deepest_state ( drv , dev , false ) ; }
bool process_record_user ( uint16_t keycode , keyrecord_t * record ) { switch ( keycode ) { case LOWER : if ( record -> event . pressed ) { layer_on ( _LOWER ) ; update_tri_layer ( _LOWER , _RAISE , _ADJUST ) ; } else { layer_off ( _LOWER ) ; update_tri_layer ( _LOWER , _RAISE , _ADJUST ) ; } return false ; break ; case 

In [ ]:
print("  lines,  words,  characters the src-all.txt contains ")
!wc -mlw src-all.txt
print("  lines,  words,  characters the tgt-all.txt contains ")
!wc -mlw tgt-all.txt

  lines,  words,  characters the src-all.txt contains 
    643695  356284731 2060915253 src-all.txt
  lines,  words,  characters the tgt-all.txt contains 
    643695  360890953 2091037112 tgt-all.txt


## 进行BPE preprocess

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
with open('/content/data/src-all.txt','r') as f:
  source = f.read()
uncleaned_source_list = source.split('\n')
print("length for list of uncleaned source")
print(len(uncleaned_source_list))

with open('/content/data/tgt-all.txt','r') as f:
  target = f.read()
uncleaned_target_list = target.split('\n')
print("length for list of uncleaned target")
print(len(uncleaned_target_list))

length for list of uncleaned source
643696
length for list of uncleaned target
643696


In [ ]:
X = uncleaned_source_list
Y = uncleaned_target_list
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.2)

In [ ]:
%%bash
mkdir -p gpt2_bpe
wget -O gpt2_bpe/encoder.json https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
wget -O gpt2_bpe/vocab.bpe https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe

--2020-11-12 03:19:24--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘gpt2_bpe/encoder.json’

     0K .......... .......... .......... .......... ..........  4%  223K 4s
    50K .......... .......... .......... .......... ..........  9% 1.38M 2s
   100K .......... .......... .......... .......... .......... 14%  605K 2s
   150K .......... .......... .......... .......... .......... 19% 12.8M 1s
   200K .......... .......... .......... .......... .......... 24%  465K 1s
   250K .......... .......... .......... .......... .......... 29% 14.8M 1s
   300K .......... .......... .......... .......... .......... 34% 10.5M 1s
   350K .......... .......... .......... .......... ........

### 使用sentencepiece 进行BPE有关的操作

In [ ]:
# 好像这个不需要
%%bash
sudo apt-get install cmake build-essential pkg-config libgoogle-perftools-dev

Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.4ubuntu1).
pkg-config is already the newest version (0.29.1-0ubuntu2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
The following additional packages will be installed:
  libunwind-dev
The following NEW packages will be installed:
  libgoogle-perftools-dev libunwind-dev
0 upgraded, 2 newly installed, 0 to remove and 12 not upgraded.
Need to get 627 kB of archives.
After this operation, 6,761 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libunwind-dev amd64 1.2.1-8 [423 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgoogle-perftools-dev amd64 2.5-2.2ubuntu3 [204 kB]
Fetched 627 kB in 2s (281 kB/s)
Selecting previously unselected package libunwind-dev:amd64.
(Reading database ... 144786 files and directories currently installed.)
Preparing to unpack .../libunwind-dev_1.2.

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [ ]:
!git clone https://github.com/google/sentencepiece.git

Cloning into 'sentencepiece'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 3451 (delta 95), reused 92 (delta 39), pack-reused 3273
Receiving objects: 100% (3451/3451), 28.50 MiB | 23.37 MiB/s, done.
Resolving deltas: 100% (2373/2373), done.


In [ ]:
%%bash
cd /content/sentencepiece
mkdir build
cd build
cmake ..
make -j $(nproc)
sudo make install
sudo ldconfig -v

-- VERSION: 0.1.94
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Not Found TCMalloc: TCMALLOC_LIB-NOTFOUND
-

/sbin/ldconfig.real: Path `/usr/local/lib' given more than once
/sbin/ldconfig.real: Can't stat /usr/local/nvidia/lib: No such file or directory
/sbin/ldconfig.real: Can't stat /usr/local/nvidia/lib64: No such file or directory
/sbin/ldconfig.real: Can't stat /usr/local/lib/x86_64-linux-gnu: No such file or directory
/sbin/ldconfig.real: Path `/lib/x86_64-linux-gnu' given more than once
/sbin/ldconfig.real: Path `/usr/lib/x86_64-linux-gnu' given more than once
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

/sbin/ldconfig.real: /lib/x86_64-linux-gnu/ld-2.27.so is the dynamic linker, ignoring

/sbin/ldconfig.real: /lib32/ld-2.27.so is the dynamic linker, ignoring



In [ ]:
!spm_train --input=/content/data/src-all.txt --model_prefix=data --vocab_size=16000 --character_coverage=1.0 --model_type=bpe --max_sentence_length=12000

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/data/src-all.txt
  input_format: 
  model_prefix: data
  model_type: BPE
  vocab_size: 16000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 12000
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  escape_whitespaces: 1
  normalization_ru

In [ ]:
!spm_encode --model=data.model --output_format=piece /content/data/src-all.txt > srcoutput

In [ ]:
!spm_train --input=/content/data/tgt-all.txt --model_prefix=source --vocab_size=16000 --character_coverage=1.0 --model_type=bpe --max_sentence_length=12000

In [ ]:
!spm_encode --model=source.model --output_format=piece /content/data/tgt-all.txt > tgtoutput

In [ ]:
!head -n 10 '/content/srcoutput'

▁void ▁cpu idle _ enter _ freeze ▁( ▁void ▁) ▁{ ▁struct ▁cpu idle _ device ▁* ▁dev ▁= ▁__ this _ cpu _ read ▁( ▁cpu idle _ devices ▁) ▁; ▁struct ▁cpu idle _ driver ▁* ▁drv ▁= ▁cpu idle _ get _ cpu _ driver ▁( ▁dev ▁) ▁; ▁int ▁index ▁; ▁if ▁( ▁cpu idle _ not _ available ▁( ▁drv ▁, ▁dev ▁) ▁) ▁goto ▁fallback ▁; ▁index ▁= ▁cpu idle _ find _ de e pe st _ state ▁( ▁drv ▁, ▁dev ▁, ▁true ▁) ▁; ▁if ▁( ▁index ▁>= ▁0 ▁) ▁{ ▁enter _ freeze _ pro per ▁( ▁drv ▁, ▁dev ▁, ▁index ▁) ▁; ▁local _ irq _ enable ▁( ▁) ▁; ▁return ▁; ▁} ▁index ▁= ▁cpu idle _ find _ de e pe st _ state ▁( ▁drv ▁, ▁dev ▁, ▁false ▁) ▁; ▁if ▁( ▁index ▁>= ▁0 ▁) ▁{ ▁cpu idle _ enter ▁( ▁drv ▁, ▁dev ▁, ▁index ▁) ▁; ▁return ▁; ▁} ▁fallback ▁: ▁arch _ cpu _ idle ▁( ▁) ▁; ▁}
▁int ▁cpu idle _ select ▁( ▁struct ▁cpu idle _ driver ▁* ▁drv ▁, ▁struct ▁cpu idle _ device ▁* ▁dev ▁) ▁{ ▁if ▁( ▁cpu idle _ not _ available ▁( ▁drv ▁, ▁dev ▁) ▁) ▁return ▁- ▁ENODEV ▁; ▁return ▁cpu idle _ curr _ governor ▁-> ▁select ▁( ▁drv ▁, ▁dev ▁) ▁; ▁}
▁static

In [ ]:
#把数据拷贝到google云盘中
# !cp '/content/tgtoutput.txt' -r '/content/drive/My Drive/bugData/'
!cp '/content/srcoutput' -r '/content/drive/My Drive/bugData/'

## fairseq 安装
- preprocess使用到fairseq preprocess

In [ ]:
%%bash
git clone https://github.com/pytorch/fairseq
cd /content/fairseq
pip install --editable ./

Obtaining file:///content/fairseq
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141230 sha256=6d2d2297a02e1802f47fe92786cdb18c1da987ed8d6239cffb8b6b3b4af23f89
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=ffca9520349efcf3d9219a6ab1f420b797b740972405b2d84abae8910248dd5c
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b25108

Cloning into 'fairseq'...


In [ ]:
%%bash
in="/content/data/srcoutput"
train="/content/data/train.buggy"
test="/content/data/test.buggy"
awk -v train="$train" -v test="$test" '{if(rand()<0.9) {print > train} else {print > test}}' $in

In [14]:
%%bash
in="/content/data/targetoutput.txt"
train="/content/data/train.repaired"
test="/content/data/test.repaired"
awk -v train="$train" -v test="$test" '{if(rand()<0.9) {print > train} else {print > test}}' $in

In [ ]:
#把一个文件分为多个文件来加速有关的训练
%%bash
cd data
mkdir AfterSplit
cd AfterSplit 
split -l 50000 -d --additional-suffix=.buggy "/content/data/srcoutput.train"

### inspect inside the data
- 仔细查看train和srcoutput的具体数据的处理情况

In [ ]:
!head -n 10 /content/data/srcoutput.train

▁void ▁cpu idle _ enter _ freeze ▁( ▁void ▁) ▁{ ▁struct ▁cpu idle _ device ▁* ▁dev ▁= ▁__ this _ cpu _ read ▁( ▁cpu idle _ devices ▁) ▁; ▁struct ▁cpu idle _ driver ▁* ▁drv ▁= ▁cpu idle _ get _ cpu _ driver ▁( ▁dev ▁) ▁; ▁int ▁index ▁; ▁if ▁( ▁cpu idle _ not _ available ▁( ▁drv ▁, ▁dev ▁) ▁) ▁goto ▁fallback ▁; ▁index ▁= ▁cpu idle _ find _ de e pe st _ state ▁( ▁drv ▁, ▁dev ▁, ▁true ▁) ▁; ▁if ▁( ▁index ▁>= ▁0 ▁) ▁{ ▁enter _ freeze _ pro per ▁( ▁drv ▁, ▁dev ▁, ▁index ▁) ▁; ▁local _ irq _ enable ▁( ▁) ▁; ▁return ▁; ▁} ▁index ▁= ▁cpu idle _ find _ de e pe st _ state ▁( ▁drv ▁, ▁dev ▁, ▁false ▁) ▁; ▁if ▁( ▁index ▁>= ▁0 ▁) ▁{ ▁cpu idle _ enter ▁( ▁drv ▁, ▁dev ▁, ▁index ▁) ▁; ▁return ▁; ▁} ▁fallback ▁: ▁arch _ cpu _ idle ▁( ▁) ▁; ▁}
▁int ▁cpu idle _ select ▁( ▁struct ▁cpu idle _ driver ▁* ▁drv ▁, ▁struct ▁cpu idle _ device ▁* ▁dev ▁) ▁{ ▁if ▁( ▁cpu idle _ not _ available ▁( ▁drv ▁, ▁dev ▁) ▁) ▁return ▁- ▁ENODEV ▁; ▁return ▁cpu idle _ curr _ governor ▁-> ▁select ▁( ▁drv ▁, ▁dev ▁) ▁; ▁}
▁static

In [ ]:
!head -n 10 /content/data/srcoutput

▁void ▁cpu idle _ enter _ freeze ▁( ▁void ▁) ▁{ ▁struct ▁cpu idle _ device ▁* ▁dev ▁= ▁__ this _ cpu _ read ▁( ▁cpu idle _ devices ▁) ▁; ▁struct ▁cpu idle _ driver ▁* ▁drv ▁= ▁cpu idle _ get _ cpu _ driver ▁( ▁dev ▁) ▁; ▁int ▁index ▁; ▁if ▁( ▁cpu idle _ not _ available ▁( ▁drv ▁, ▁dev ▁) ▁) ▁goto ▁fallback ▁; ▁index ▁= ▁cpu idle _ find _ de e pe st _ state ▁( ▁drv ▁, ▁dev ▁, ▁true ▁) ▁; ▁if ▁( ▁index ▁>= ▁0 ▁) ▁{ ▁enter _ freeze _ pro per ▁( ▁drv ▁, ▁dev ▁, ▁index ▁) ▁; ▁local _ irq _ enable ▁( ▁) ▁; ▁return ▁; ▁} ▁index ▁= ▁cpu idle _ find _ de e pe st _ state ▁( ▁drv ▁, ▁dev ▁, ▁false ▁) ▁; ▁if ▁( ▁index ▁>= ▁0 ▁) ▁{ ▁cpu idle _ enter ▁( ▁drv ▁, ▁dev ▁, ▁index ▁) ▁; ▁return ▁; ▁} ▁fallback ▁: ▁arch _ cpu _ idle ▁( ▁) ▁; ▁}
▁int ▁cpu idle _ select ▁( ▁struct ▁cpu idle _ driver ▁* ▁drv ▁, ▁struct ▁cpu idle _ device ▁* ▁dev ▁) ▁{ ▁if ▁( ▁cpu idle _ not _ available ▁( ▁drv ▁, ▁dev ▁) ▁) ▁return ▁- ▁ENODEV ▁; ▁return ▁cpu idle _ curr _ governor ▁-> ▁select ▁( ▁drv ▁, ▁dev ▁) ▁; ▁}
▁static

### use fairseq-train to train the model
- 使用fairseq train来训练整个模型

In [17]:
%%bash
cd fairseq
mkdir data-bin

In [ ]:
%%bash
TEXT=/content/data
fairseq-preprocess --source-lang buggy --target-lang repaired \
    --trainpref $TEXT/train \
    --testpref $TEXT/test \
    --destdir /content/fairseq/data-bin

In [16]:
%%bash
cd fairseq
mkdir -p checkpoints/fconv
CUDA_VISIBLE_DEVICES=0 python train.py /content/fairseq/data-bin \
  --lr 0.25 --clip-norm 0.1 --dropout 0.2 --max-tokens 4000 \
  --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
  --lr-scheduler fixed --force-anneal 200 \
  --arch roberta_base --save-dir checkpoints/fconv \
  --skip-invalid-size-inputs-valid-test --max-epoch 20

2020-11-17 08:56:09 | INFO | fairseq_cli.train | {'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False}, 'common_eval': {'_name': None, 'path': None, 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_port': -1, 'device_id': 0, 'local_rank': 0, 'distribut

Traceback (most recent call last):
  File "train.py", line 14, in <module>
    cli_main()
  File "/content/fairseq/fairseq_cli/train.py", line 392, in cli_main
    distributed_utils.call_main(cfg, main)
  File "/content/fairseq/fairseq/distributed_utils.py", line 334, in call_main
    main(cfg, **kwargs)
  File "/content/fairseq/fairseq_cli/train.py", line 66, in main
    task = tasks.setup_task(cfg.task)
  File "/content/fairseq/fairseq/tasks/__init__.py", line 44, in setup_task
    return task.setup_task(cfg, **kwargs)
  File "/content/fairseq/fairseq/tasks/translation.py", line 262, in setup_task
    "Could not infer language pair, please provide it explicitly"
Exception: Could not infer language pair, please provide it explicitly


### 使用fairseq来验证是否训练正确
- fairseq generate